# Question 2,3: Design and describe an architecture for CGAN 

Train CGAN using MNIST
training examples (60K) and perform evaluation experiments.
Try different architectures, hyper-parameters, and, if necessary, the aspects of one-sided label
smoothing, virtual batch normalization, balancing G and D.
Please perform qualitative analyses on the generated images, and discuss, with results, what
challenge and how they are specifically addressing. Is there the mode collapse issue? 

Measure the inception scores i.e. we use the class labels to generate images in CGAN and compare them with the predicted labels of the generated images by a pre-trained classifier, measuring the success rate
The pre-trained classifier is a classification neural network with including convolutional layers and a softmax layer, using the MNIST training set (60K). Also report the recognition accuracies on the MNIST real testing set (10K), in comparison to the inception scores. Please measure and discuss the inception scores for the different hyper-parameters/tricks and/or architectures in Q2.

## Import of Modules

In [1]:
import sys, json, os, glob, time, cv2, random
import keras
from keras import backend as K
from keras.utils import plot_model, np_utils, to_categorical

import tensorflow as tf
import numpy as np
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

from load_data import *
from cgan import *
from post_process import *
from inception_model import *
import json
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


## Fix Seeds

In [2]:
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

## Load Data

In [3]:
X_train, Y_train, X_test, Y_test = load_data()
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

## Load Inception Network

In [4]:
inception_model = LeNet()
if os.path.isfile("models/LeNet.h5"):
    inception_model.load_model("models/LeNet")
else:
    inception_model.train(X_train, Y_train, 5, batch_size=128, save=True, X_test=X_test, y_test=Y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [5]:
print(inception_model.evaluate(X_train,Y_train))
print(inception_model.evaluate(X_test,Y_test))

[0.018172569799636645, 0.9943833333333333]
[0.03675771964562882, 0.9882]


## Experiments
We define the hyperparameters that we are going to change:

1. Filter capacity can be 1,2,3 than the original
2. The ratio of discriminator to generator 1:1, 2:1, 3:1
3. The batch size being 64, 128, 256

Then after finding the best model we are going to try and use label smoothing and virtual batch normalization to see what is going to be the outcome. 

In [8]:
filters =[1,2,3]
depth = [1,2,3]
batch_size = [32, 64, 128]
epochs = 25
train = False

In [9]:
models = {}
for f in filters:
    for d in depth:
        for b in batch_size:
            # We run out of memory for these models 
            if (d ==3 and f ==3) or (b == 128 and f == 3 and d == 3) or (f==3 and (b==64 or b == 128) and d ==2):
                    continue
            name = "CGAN_{}_{}_{}".format(f,d,b)
            history = None
            model = CGAN(name, d, f)
            if train:
                history = model.train(X_train, Y_train, inception_model,
                                      epochs=epochs, batch_size = b,
                                      sample_interval= 1,
                                      save = True)
                models[name] = history
            else:
                models[name] = model.load_model("models/{}".format(name))
            # Empty the memory
            del model

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 110)          0           input_4[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 1024)         113664      concatenate_3[0][0]              
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 784)          8624        input_9[0][0]                    
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
reshape_6 (Reshape)             (None, 28, 28, 1)    0           dense_15[0][0]                   
__________________________________________________________________________________________________
concatenat

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_9 (Concatenate)     (None, 110)          0           input_13[0][0]                   
                                                                 input_15[0][0]                   
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 1024)         113664      concatenate_9[0][0]              
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_11 (Concatenate)    (None, 110)          0           input_16[0][0]                   
                                                                 input_18[0][0]                   
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 1024)         113664      concatenate_11[0][0]             
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_13 (Concatenate)    (None, 110)          0           input_19[0][0]                   
                                                                 input_21[0][0]                   
__________________________________________________________________________________________________
dense_33 (Dense)                (None, 1024)         113664      concatenate_13[0][0]             
__________

...

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_52 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_54 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_35 (Concatenate)    (None, 110)          0           input_52[0][0]                   
                                                                 input_54[0][0]                   
__________________________________________________________________________________________________
dense_88 (Dense)                (None, 1024)         113664      concatenate_35[0][0]             
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_55 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_57 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_37 (Concatenate)    (None, 110)          0           input_55[0][0]                   
                                                                 input_57[0][0]                   
__________________________________________________________________________________________________
dense_93 (Dense)                (None, 1024)         113664      concatenate_37[0][0]             
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_58 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_60 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_39 (Concatenate)    (None, 110)          0           input_58[0][0]                   
                                                                 input_60[0][0]                   
__________________________________________________________________________________________________
dense_98 (Dense)                (None, 1024)         113664      concatenate_39[0][0]             
__________

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_63 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
dense_105 (Dense)               (None, 784)          8624        input_63[0][0]                   
__________________________________________________________________________________________________
input_62 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
reshape_42 (Reshape)            (None, 28, 28, 1)    0           dense_105[0][0]                  
__________________________________________________________________________________________________
concatenat

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_67 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_69 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_45 (Concatenate)    (None, 110)          0           input_67[0][0]                   
                                                                 input_69[0][0]                   
__________________________________________________________________________________________________
dense_113 (Dense)               (None, 1024)         113664      concatenate_45[0][0]             
__________

In [10]:
discriminator_losses = []
discriminator_accuracies = []
generator_losses = []
inception_scores = []
names = []
for name, data in models.items():
    names.append(name)
    discriminator_losses.append(data['discriminator_loss'])
    discriminator_accuracies.append(data['discriminator_accuracy'])
    generator_losses.append(data['generator_loss'])
    inception_scores.append(data['inception_score'])
    
plot_graphs(discriminator_losses, discriminator_accuracies, generator_losses, names, "../../Figures/CW2/Q2/normal/", inception_scores)

## Virtual Batch Normalisation
Now we are going to apply virtual batch normalisation to the most successful model

In [ ]:
f = 1
b = 64
d = 1
epochs = 25
name = "CGAN_{}_{}_{}_vb".format(f,d,b)
history = None
model = CGAN(name, d, f)
model.virtual_batch_norm = True
train = False
if train:
    history = model.train(X_train, Y_train, inception_model,
                          epochs=epochs, batch_size = b,
                          sample_interval= 5,
                          save = True)
else:
    history = model.load_model("models/{}".format(name))
# Empty the memory
del model

discriminator_losses = [list(history['discriminator_loss'])]
discriminator_accuracies = [list(history['discriminator_accuracy'])]
generator_losses = [list(history['generator_loss'])]
inception_scores = [list(history['inception_score'])]
names = [name]

plot_graphs(discriminator_losses, discriminator_accuracies, generator_losses, names,"../../Figures/CW2/Q2/vb/", inception_scores)

## Label smoothing
Now we are going to again apply label smoothing to the most successful model

In [ ]:
f = 1
b = 64
d = 1
epochs = 25
name = "CGAN_{}_{}_{}_ls".format(f,d,b)
history = None
model = CGAN(name, d, f)
model.label_smoothing = True
train = False
if train:
    history = model.train(X_train, Y_train, inception_model,
                          epochs=epochs, batch_size = b,
                          sample_interval= 5,
                          save = True)
else:
    history = model.load_model("models/{}".format(name))
# Empty the memory
del model

discriminator_losses = [list(history['discriminator_loss'])]
discriminator_accuracies = [list(history['discriminator_accuracy'])]
generator_losses = [list(history['generator_loss'])]
inception_scores = [list(history['inception_score'])]
names = [name]

plot_graphs(discriminator_losses, discriminator_accuracies, generator_losses, names,"../../Figures/CW2/Q2/ls/", inception_scores)

## Plot confusion matrices

In [ ]:
f = 1
b = 64
d = 1
epochs = 25
name = "CGAN_{}_{}_{}_ls".format(f,d,b)
model = CGAN(name, d, f)
model.label_smoothing = True
model.virtual_batch_norm = False
train = False

noise_labels = []
for i in range(10):
    noise_labels+=[i]*100
_noise_labels = to_categorical(noise_labels, 10)

if model.label_smoothing:
    for row in _noise_labels:
        i = np.argmax(row)
        row[i] = i / 10

if train:
    _ = model.train(X_train, Y_train, inception_model,
                          epochs=epochs, batch_size = b,
                          sample_interval= 5,
                          save = True)
else:
    _ = model.load_model("models/{}".format(name), False)

generated_images = model.generate(_noise_labels)
_predicted_labels = inception_model.predict(generated_images)
predicted_labels = np.argmax(_predicted_labels)


plot_confusion_matrix(noise_labels, predicted_labels,normalize=True, title='Confusion matrix',
                     path ="../../Figures/CW2/Q2/cm_n_{}.png".format(name))
del model
